# Excercises 
# 1. Tune the network
Run the experiment below, explore the different parameters (see suggestions below) and study the result with tensorboard. 
Make a single page (1 a4) report of your findings. Use your visualisation skills to communicate your most important findings.

In [ ]:
from mads_datasets import DatasetFactoryProvider, DatasetType

from mltrainer.preprocessors import BasePreprocessor
from mltrainer import imagemodels, Trainer, TrainerSettings, ReportTypes, metrics

import torch.optim as optim
from torch import nn
from tomlserializer import TOMLSerializer

We will be using `tomlserializer` to easily keep track of our experiments, and to easily save the different things we did during our experiments.
It can export things like settings and models to a simple `toml` file, which can be easily shared, checked and modified.

First, we need the data. 

In [ ]:
fashionfactory = DatasetFactoryProvider.create_factory(DatasetType.FASHION)
preprocessor = BasePreprocessor()
streamers = fashionfactory.create_datastreamer(batchsize=64, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]
trainstreamer = train.stream()
validstreamer = valid.stream()

We need a way to determine how well our model is performing. We will use accuracy as a metric.

In [ ]:
accuracy = metrics.Accuracy()

You can set up a single experiment.

- We will show the model batches of 64 images, 
- and for every epoch we will show the model 100 batches (trainsteps=100).
- then, we will test how well the model is doing on unseen data (teststeps=100).
- we will report our results during training to tensorboard, and report all configuration to a toml file.
- we will log the results into a directory called "modellogs", but you could change this to whatever you want.

In [ ]:
import torch
loss_fn = torch.nn.CrossEntropyLoss()

settings = TrainerSettings(
    epochs=3,
    metrics=[accuracy],
    logdir="modellogs",
    train_steps=100,
    valid_steps=100,
    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
)


We will use a very basic model: a model with three linear layers.

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, num_classes: int, units1: int, units2: int) -> None:
        super().__init__()
        self.num_classes = num_classes
        self.units1 = units1
        self.units2 = units2
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, units1),
            nn.ReLU(),
            nn.Linear(units1, units2),
            nn.ReLU(),
            nn.Linear(units2, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork(
    num_classes=10, units1=256, units2=256)

I developped the `tomlserializer` package, it is a useful tool to save configs, models and settings as a tomlfile; that way it is easy to track what you changed during your experiments.

This package will 1. check if there is a `__dict__` attribute available, and if so, it will use that to extract the parameters that do not start with an underscore, like this:

In [ ]:
{k: v for k, v in model.__dict__.items() if not k.startswith("_")}

This means that if you want to add more parameters to the `.toml` file, eg `units3`, you can add them to the class like this:

```python
class NeuralNetwork(nn.Module):
    def __init__(self, num_classes: int, units1: int, units2: int, units3: int) -> None:
        super().__init__()
        self.num_classes = num_classes
        self.units1 = units1
        self.units2 = units2
        self.units3 = units3  # <-- add this line
```

And then it will be added to the `.toml` file. Check the result for yourself by using the `.save()` method of the `TomlSerializer` class like this:

In [ ]:
tomlserializer = TOMLSerializer()
tomlserializer.save(settings, "settings.toml")
tomlserializer.save(model, "model.toml")

Check the `settings.toml` and `model.toml` files to see what is in there.

You can use the `Trainer` class from my `mltrainer` module to train your model. It has the TOMLserializer integrated, so it will automatically save the settings and model to a toml file if you have added `TOML` as a reporttype in the settings.

In [ ]:
trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=loss_fn,
    optimizer=optim.Adam,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau
)
trainer.loop()

Now, check in the modellogs directory the results of your experiment.

We can now loop this with a naive approach, called a grid-search (why do you think i call it naive?).

In [ ]:
# units = [256, 128, 64]
# for unit1 in units:
#     for unit2 in units:
#         print(f"Units: {unit1}, {unit2}")

Of course, this might not be the best way to search for a model; some configurations will be better than others (can you predict up front what will be the best configuration?).

So, feel free to improve upon the gridsearch by adding your own logic.

In [ ]:
# import torch

# units = [256, 128, 64]
# loss_fn = torch.nn.CrossEntropyLoss()

# settings = TrainerSettings(
#     epochs=3,
#     metrics=[accuracy],
#     logdir="modellogs",
#     train_steps=len(train),
#     valid_steps=len(valid),
#     reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
# )

# for unit1 in units:
#     for unit2 in units:

#         model = NeuralNetwork(num_classes=10, units1=unit1, units2=unit2)

#         trainer = Trainer(
#             model=model,
#             settings=settings,
#             loss_fn=loss_fn,
#             optimizer=optim.Adam,
#             traindataloader=trainstreamer,
#             validdataloader=validstreamer,
#             scheduler=optim.lr_scheduler.ReduceLROnPlateau
#         )
#         trainer.loop()


In [ ]:
class NeuralNetworkBigger(nn.Module):
    def __init__(self, num_classes: int, units: list, activation: str = "relu") -> None:
        super().__init__()
        self.num_classes = num_classes
        self.units = units
        self.activation = activation
        self.flatten = nn.Flatten()
        layers = []
        input_dim = 28 * 28

        # Choose activation function
        if activation == "relu":
            act_fn = nn.ReLU()
        elif activation == "tanh":
            act_fn = nn.Tanh()
        elif activation == "sigmoid":
            act_fn = nn.Sigmoid()
        else:
            raise ValueError(f"Unsupported activation: {activation}")

        for unit in units:
            layers.append(nn.Linear(input_dim, unit))
            layers.append(act_fn)
            input_dim = unit
        layers.append(nn.Linear(input_dim, num_classes))
        self.linear_relu_stack = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Because we have set the ReportType to TOML, you will find in every log dir a model.toml and settings.toml file.

Run the experiment, and study the result with tensorboard. 

Locally, it is easy to do that with VS code itself. On the server, you have to take these steps:

- in the terminal, `cd` to the location of the repository
- activate the python environment for the shell. Note how the correct environment is being activated.
- run `tensorboard --logdir=modellogs` in the terminal
- tensorboard will launch at `localhost:6006` and vscode will notify you that the port is forwarded
- you can either press the `launch` button in VScode or open your local browser at `localhost:6006`

In [ ]:
import torch

unit1 = 256
unit2 = 128
unit3 = 64
unit4 = 32
# unit1 = 1024
# unit2 = 512
loss_fn = torch.nn.CrossEntropyLoss()

settings = TrainerSettings(
    # epochs=5,
    epochs=10,
    # epochs=20,
    metrics=[accuracy],
    logdir="modellogs",
    train_steps=len(train),
    valid_steps=len(valid),
    # learning_rate=0.001,
    # learning_rate=0.01,
    optim="Adam",
    # train_steps=8,
    # valid_steps=2,
    # train_steps=64,
    # valid_steps=64,
    # train_steps=128,
    # # valid_steps=32,
    # valid_steps=128,


    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
)

model = NeuralNetwork(num_classes=10, units1=unit1, units2=unit2)
# model = NeuralNetworkBigger(num_classes=10, units=[unit1, unit2, unit3])
# model = NeuralNetworkBigger(num_classes=10, units=[unit1, unit2, unit3, unit4])
# model = NeuralNetworkBigger(num_classes=10, units=[unit1, unit2], activation="tanh")

trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=loss_fn,
    optimizer=optim.Adam,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau
)
trainer.loop()

# uv run tensorboard --logdir 1-hypertuning-gridsearch/modellogs/


In [ ]:
import torch

loss_fn = torch.nn.CrossEntropyLoss()

settings = TrainerSettings(
    epochs=100,
    metrics=[accuracy],
    earlystop_kwargs={"patience": 5, "verbose":True},
    logdir="experiment2",
    train_steps=len(train),
    valid_steps=len(valid),
    optim="Adam",
    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
)

layers = [2,3,4]
units = [32, 128, 512]

for layer in layers:
    for unit in units:
        model = NeuralNetworkBigger(num_classes=10, units=[unit]*layer)
        print(f"Training model with {layer} layers and {unit} units per layer")
        trainer = Trainer(
            model=model,
            settings=settings,
            loss_fn=loss_fn,
            optimizer=optim.Adam,
            traindataloader=trainstreamer,
            validdataloader=validstreamer,
            scheduler=optim.lr_scheduler.ReduceLROnPlateau
        )
        trainer.loop()


# uv run tensorboard --logdir 1-hypertuning-gridsearch/experiment/


In [ ]:
# Get the accuracy of all models
import os
import pandas as pd
results = []
for root, dirs, files in os.walk("experiment2"):
    for file in files:
        if file == "results.toml":
            filepath = os.path.join(root, file)
            data = tomlserializer.load(filepath)
            accuracy = data["valid"]["Accuracy"]
            settings = tomlserializer.load(os.path.join(root, "settings.toml"))
            model = tomlserializer.load(os.path.join(root, "model.toml"))
            units = model["units"]
            num_layers = len(units)
            num_units = units[0] if num_layers > 0 else 0
            results.append({
                "path": root,
                "accuracy": accuracy,
                "num_layers": num_layers,
                "num_units": num_units,
                "epochs": settings["epochs"],
                "optimizer": settings["optim"],
                "learning_rate": settings["learning_rate"],
            })